In [1]:
# import pprint
# from zipfile import ZipFile

# path = "./PythonTools-3.7.0-py2.py3-none-any.whl"
# names = ZipFile(path).namelist()
# pprint.pprint(names)

In [2]:
from pathlib import Path
import json

import pydicom as dicom
import numpy as np
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import slicerio # https://pypi.org/project/slicerio/

from PythonTools.rek2py import rek2py
# from PythonTools.raw2dicom import raw2dicom

In [3]:
MODELNAME = "SegFormer_shvit_2conv"
SHVIT_SEGFORMER = False
# MODELNAME = "SegFormer"
# SHVIT_SEGFORMER = False

NUM_LAYERS = 10

# DEPTH, HEIGHT, WIDTH = 128, 128, 128
# DEPTH, HEIGHT, WIDTH = 196, 196, 196
DEPTH, HEIGHT, WIDTH = 224, 224, 224
# DEPTH, HEIGHT, WIDTH = 256, 256, 256
# DEPTH, HEIGHT, WIDTH = 272, 272, 272
# DEPTH, HEIGHT, WIDTH = 288, 288, 288
# DEPTH, HEIGHT, WIDTH = 336, 336, 336
# DEPTH, HEIGHT, WIDTH = 464, 464, 464
# DEPTH, HEIGHT, WIDTH = 480, 480, 480
# DEPTH, HEIGHT, WIDTH = 512, 512, 512

factor = 4
if (SHVIT_SEGFORMER):
    factor = 16

In [4]:
def list_all_files(directory, ext):

    directory_path = Path(directory)
    # List all files in the directory and subdirectories
    all_files = list(directory_path.rglob(ext))  

    # Filter out directories
    files = [file for file in all_files if file.is_file()]
    return files


# Example usage
path_to_vol = "../../../../../Documents/Python/shoes_segformer/Data/volumes/"
directory = path_to_vol

files = list_all_files(directory, "*.rek")
files

[PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Adidas_Martin-weiss.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Adidas_Martin.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Bruschi_down2_2_2.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Camper_down2_2_2.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Citywalk-sit-taupe-34_down2_2_2.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Citywalk-sit-taupe-36_down2_2_2.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Citywalk-sit-taupe-39_down2_2_2.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Citywalk-sit-taupe-40_down2_2_2.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Citywalk-sit-taupe-42_down2_2_2.rek'),
 PosixPath('../../../../../Documents/Python/shoes_s

In [5]:
# after run check how shoes are oriented in the box; repeat only the files where shoes are not displayed from the side - modify transpose


rotate_shoes = [path_to_vol + "Adidas_Martin.rek",path_to_vol + "Citywalk-sit-taupe-34_down2_2_2.rek", 
                path_to_vol + "Citywalk-sit-taupe-36_down2_2_2.rek", path_to_vol + "Citywalk-sit-taupe-40_down2_2_2.rek", 
                path_to_vol + "Schuh_Martin_down2_2_2.rek", path_to_vol + "Sneaker_Dana_Nike.rek",
                path_to_vol + "Sneaker_Dana_Puma_Flyer.rek"]

# Convert to pathlib.Path objects
rotate_shoes = [Path(folder) for folder in rotate_shoes]
rotate_shoes


[PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Adidas_Martin.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Citywalk-sit-taupe-34_down2_2_2.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Citywalk-sit-taupe-36_down2_2_2.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Citywalk-sit-taupe-40_down2_2_2.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Schuh_Martin_down2_2_2.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Sneaker_Dana_Nike.rek'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/volumes/Sneaker_Dana_Puma_Flyer.rek')]

In [6]:
from scipy import ndimage

def resize_volume(volume, depth=DEPTH, height=HEIGHT, width=WIDTH, factor=1):
    
    (vol_depth, vol_height, vol_width) = volume.shape
    
    depth_factor = depth/vol_depth/factor
    height_factor = height/vol_height/factor
    width_factor = width/vol_width/factor
    
    print("xx"*20, volume.shape)
    print("xx"*20, depth_factor, height_factor, width_factor)
    
    volume = ndimage.zoom(volume, (depth_factor, height_factor, width_factor), order=1)
    print("xx"*20, volume.shape)
    
    return volume
    

In [7]:
def saveVolumes(folder, files):
    for file in files:
        
        # Get the file name without extension
        file_path  = Path(file)
        filename =  file_path.stem
        print(filename)

        # dicomdata = raw2dicom(image=volume, ezrt_header=header)
        header, volume = rek2py(filepath=file, switch_order=True)
        print(volume.shape)
    
        # createAnimation(filename, volume)
        if file in rotate_shoes:
            orientation = (1,2,0)
        else:
            orientation = (2,1,0)
            
        volume = volume.transpose(orientation)      
        volume = resize_volume(volume)
        
        folder_path = Path(folder)
        folder_path.mkdir(parents=True, exist_ok=True)
        file_path = f"{folder_path.joinpath(filename)}" #.npz"
        # np.savez_compressed(file_path, volume)
        np.save(file_path, volume)
    

# target_directory = f"/mnt/d/Datasets/shoes/images_3d_{MODELNAME}_{DEPTH}x{HEIGHT}x{WIDTH}/volumes"
target_directory = f"./images/images_3d_{MODELNAME}_{DEPTH}x{HEIGHT}x{WIDTH}/volumes"
saveVolumes(target_directory, files)

Adidas_Martin-weiss
(993, 988, 988)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (988, 988, 993)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.22672064777327935 0.22672064777327935 0.2255790533736153
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (224, 224, 224)
Adidas_Martin
(993, 990, 990)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (990, 990, 993)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.22626262626262628 0.22626262626262628 0.2255790533736153
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (224, 224, 224)
Bruschi_down2_2_2
(876, 650, 475)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (475, 650, 876)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.47157894736842104 0.3446153846153846 0.2557077625570776
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (224, 224, 224)
Camper_down2_2_2
(880, 675, 425)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (425, 675, 880)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.5270588235294118 0.33185185185185184 0.2545454545454545
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (224, 224, 224)
Citywalk-sit-

In [8]:
# import numpy as np
# import plotly.graph_objects as go
# import plotly.io as pio
# pio.renderers.default = "vscode"


# def plotVolume(folder, file):
    
#     # Get the file name without extension
#     file_path  = Path(file)
#     filename =  file_path.stem
#     print(filename)    
    
#     # Load 3D numpy array
#     shoe_volume = np.load(f"/mnt/d/Datasets/shoes/images_3d_{MODELNAME}_{DEPTH}x{HEIGHT}x{WIDTH}/volumes/{filename}.npy")  # Ensure this is a (128,128,128) NumPy array

#     # Create the 3D grid
#     x, y, z = np.mgrid[:DEPTH, :HEIGHT, :WIDTH]

#     # Create the volume plot
#     fig = go.Figure(data=go.Volume(
#         x=x.flatten(),
#         y=y.flatten(),
#         z=z.flatten(),
#         value=shoe_volume.flatten(),
#         opacity=0.1,  # Adjust for visibility
#         surface_count=30,  # Number of isosurfaces
#         colorscale="gray"  # Set grayscale colormap
#     ))

#     # Show the figure
#     fig.update_layout(
#         width=1000,  # Set the width in pixels
#         height=1000,   # Set the height in pixels
#         scene=dict(
#             xaxis=dict(showbackground=False),
#             yaxis=dict(showbackground=False),
#             zaxis=dict(showbackground=False),
#         )
#     )

#     filename = f"plot_{DEPTH}x{HEIGHT}x{WIDTH}_{filename}.html"
#     folder_path = Path(folder)
#     folder_path.mkdir(parents=True, exist_ok=True)
#     file_path = f"{folder_path.joinpath(filename)}"

#     print(file_path)
#     fig.write_html(file_path)
    
    
# plot_directory = f"/mnt/d/Datasets/shoes/images_3d/plots"
# volume_directory = f"/mnt/d/Datasets/shoes/images_3d_{MODELNAME}_{DEPTH}x{HEIGHT}x{WIDTH}/volumes"
# files = list_all_files(volume_directory, "*.npy")
    
# # for file in files:
# #     plotVolume(plot_directory, file)

In [9]:
path_to_ano = "../../../../../Documents/Python/shoes_segformer/Data/annotations"
directory = path_to_ano
files = list_all_files(directory, "*.seg.nrrd")

for file in files:
    
    # Get the file name without extension
    file_path  = Path(file)
    filename =  file_path.stem
    print(filename)    

    segmentation = slicerio.read_segmentation(str(file), skip_voxels=True)

    number_of_segments = len(segmentation["segments"])
    print(f"Number of segments: {number_of_segments}")

    segment_names = slicerio.segment_names(segmentation)
    print(f"Segment names: {', '.join(segment_names)}")

Adidas_Martin-weiss.seg
Number of segments: 11
Segment names: Background, Schuh_1, Schuh_2, Karton, Innenvolumen_1, Innenvolumen_2, Fllmaterial, Innensohle, Obermaterial, Auensohle, Zunge
Adidas_Martin.seg
Number of segments: 11
Segment names: Background, Karton, Schuh_1, Schuh_2, Innenvolumen_1, Innenvolumen_2, Zunge, Innensohle, Fllmaterial, Obermaterial, Auensohle
Bruschi_down2_2_2.seg
Number of segments: 12
Segment names: Background, Schuh_1, Schuh_2, Karton, Innenvolumen_1, Innenvolumen_2, Obermaterial, Innensohle, Auensohle, Fllmaterial, Zunge, Dummyschuh
Camper_down2_2_2.seg
Number of segments: 11
Segment names: Background, Schuh_1, Schuh_2, Karton, Innenvolumen_1, Innenvolumen_2, Obermaterial, Innensohle, Auensohle, Fllmaterial, Zunge
Citywalk 34.seg
Number of segments: 11
Segment names: Background, Schuh_1, Schuh_2, Karton, Innenvolumen_1, Innenvolumen_2, Obermaterial, Innensohle, Auensohle, Fllmaterial, Zunge
Citywalk 36.seg
Number of segments: 11
Segment names: Background, S

## Extract image and masks for layer xyz of all shoes

In [10]:
rotate_shoes = [path_to_ano + "Adidas_Martin.seg.nrrd", path_to_ano + "Citywalk 34.seg.nrrd", 
                path_to_ano + "Citywalk 36.seg.nrrd", path_to_ano + "Citywalk 40.seg.nrrd", 
                path_to_ano + "Schuh_Martin_down2_2_2.seg.nrrd", path_to_ano + "Sneaker_Dana_Nike.seg.nrrd", 
                path_to_ano + "Sneaker_Dana_Puma_Flyer.seg.nrrd"]

rotate_shoes = [Path(folder) for folder in rotate_shoes]

class_dict = {"Karton": 1, "Außensohle": 2, "Innensohle": 3, "Obermaterial": 4, "Zunge": 5, "Füllmaterial": 6}

In [11]:
key = "Zunge"
class_dict[key]

5

In [12]:
path_to_ano + "Adidas_Martin.seg.nrrd" in rotate_shoes

False

In [13]:
def extractMask(key, volume):

    N = 1 # remove N pixels from the border

    # Remove N pixels from the border because some are black even when the image is very bright; otherwise normalization (min/max) give not so good results  (unfortunately, not all images can be corrected ...)
    #print("shape before: ", vol.shape)
    #vol = vol[N:-N, N:-N]
    #print("shape after: ", vol.shape)

    # Normalize the volume data to the range 0-255
    normalized_volume = (volume - volume.min()) / (volume.max() - volume.min() + 1e-5) * class_dict[key]
    normalized_volume = normalized_volume.astype(np.uint8)      
    return normalized_volume

In [14]:
# tmp
def list_all_files(directory, ext):

    directory_path = Path(directory)
    # List all files in the directory and subdirectories
    all_files = list(directory_path.rglob(ext))  

    # Filter out directories
    files = [file for file in all_files if file.is_file()]
    return files



In [15]:
# ../../../../../Documents/Python/shoes_segformer/Data/annotations/Adidas_Martin-weiss.seg.nrrd
list_all_files("../../../../../Documents/Python/shoes_segformer/Data/annotations/", "*.seg.nrrd")

[PosixPath('../../../../../Documents/Python/shoes_segformer/Data/annotations/Adidas_Martin-weiss.seg.nrrd'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/annotations/Adidas_Martin.seg.nrrd'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/annotations/Bruschi_down2_2_2.seg.nrrd'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/annotations/Camper_down2_2_2.seg.nrrd'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/annotations/Citywalk 34.seg.nrrd'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/annotations/Citywalk 36.seg.nrrd'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/annotations/Citywalk 39.seg.nrrd'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/annotations/Citywalk 40.seg.nrrd'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/annotations/Citywalk 42.seg.nrrd'),
 PosixPath('../../../../../Documents/Python/shoes_segformer/Data/annot

In [16]:
def createMask(folder, file):

    segmentation_info = slicerio.read_segmentation(file)
    segment_names = slicerio.segment_names(segmentation_info) # Get semgnet_names where 3D Slicer

    segment_names_to_labels = []
    for j, sn in enumerate(segment_names):
        segment_names_to_labels.append((sn, j+1)) 
        if not segment_names_to_labels:
            continue
        
    # Numbering according to Segmented labels of 3D Slicer
    data_Karton         = slicerio.extract_segments(segmentation_info, [[item for item in segment_names_to_labels if item[0] == "Karton"][0]])
    data_Aussensohle    = slicerio.extract_segments(segmentation_info, [[item for item in segment_names_to_labels if item[0] == "Auensohle"][0]])
    data_Innensohle     = slicerio.extract_segments(segmentation_info, [[item for item in segment_names_to_labels if item[0] == "Innensohle"][0]])
    data_Obermaterial   = slicerio.extract_segments(segmentation_info, [[item for item in segment_names_to_labels if item[0] == "Obermaterial"][0]])
    data_Zunge          = slicerio.extract_segments(segmentation_info, [[item for item in segment_names_to_labels if item[0] == "Zunge"][0]])
    data_Fuellmaterial  = slicerio.extract_segments(segmentation_info, [[item for item in segment_names_to_labels if item[0] == "Fllmaterial"][0]])

    if file in rotate_shoes:
        orientation = (1,0,2) # these shoes need different orientation / transpose
    else:
        orientation = (0,1,2) # normal orientation for all shoes
            
    print(orientation)
    data_Karton        = data_Karton.get("voxels").transpose(orientation)            
    data_Aussensohle   = data_Aussensohle.get("voxels").transpose(orientation)            
    data_Obermaterial  = data_Obermaterial.get("voxels").transpose(orientation)            
    data_Innensohle    = data_Innensohle.get("voxels").transpose(orientation)            
    data_Zunge         = data_Zunge.get("voxels").transpose(orientation)            
    data_Fuellmaterial = data_Fuellmaterial.get("voxels").transpose(orientation)            
        
    # print(np.unique(data_Karton))
    # print(np.unique(data_Aussensohle))
    # print(np.unique(data_Obermaterial))
    # print(np.unique(data_Innensohle))
    # print(np.unique(data_Zunge))
    # print(np.unique(data_Fuellmaterial))
    
    data_Karton = resize_volume(data_Karton, factor=factor)
    data_Aussensohle = resize_volume(data_Aussensohle, factor=factor)
    data_Obermaterial = resize_volume(data_Obermaterial, factor=factor)
    data_Innensohle = resize_volume(data_Innensohle, factor=factor)
    data_Zunge = resize_volume(data_Zunge, factor=factor)
    data_Fuellmaterial = resize_volume(data_Fuellmaterial, factor=factor)


    # Create an empty volume filled with zeros
    final_volume = np.zeros(data_Karton.shape, dtype=np.uint8)
    final_volume[data_Karton>0] = class_dict["Karton"]
    final_volume[data_Aussensohle>0] = class_dict["Außensohle"]
    final_volume[data_Innensohle>0] = class_dict["Innensohle"]
    final_volume[data_Obermaterial>0] = class_dict["Obermaterial"]
    final_volume[data_Zunge>0] = class_dict["Zunge"]
    final_volume[data_Fuellmaterial>0] = class_dict["Füllmaterial"]

    print(final_volume.shape, np.unique(final_volume))
    
    
    if file.suffix:  # Check again if there's at least one suffix
        filename =  file.stem

        if file.suffix:  # Check again if there's at least one suffix
            filename = filename.rsplit('.', 1)[0]  # Remove the last part after the last '.'
    print(filename)  
    
    folder_path = Path(folder)
    folder_path.mkdir(parents=True, exist_ok=True)
    file_path = f"{folder_path.joinpath(filename)}" #.npz"
    # np.savez_compressed(file_path, volume)
    np.save(file_path, final_volume)

        
    
    
# extract mask information  
target_directory = f"./images/images_3d_{MODELNAME}_{DEPTH}x{HEIGHT}x{WIDTH}/masks"
path_to_ano = path_to_ano 

files = list_all_files(path_to_ano, "*.seg.nrrd")
for file in files[-4:]:
    print(file)
    createMask(target_directory, file)   

../../../../../Documents/Python/shoes_segformer/Data/annotations/TamarisPump 38.seg.nrrd
(0, 1, 2)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (968, 968, 1129)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.05785123966942149 0.05785123966942149 0.049601417183348095
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (56, 56, 56)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (968, 968, 1129)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.05785123966942149 0.05785123966942149 0.049601417183348095
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (56, 56, 56)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (968, 968, 1129)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.05785123966942149 0.05785123966942149 0.049601417183348095
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (56, 56, 56)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (968, 968, 1129)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.05785123966942149 0.05785123966942149 0.049601417183348095
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (56, 56, 56)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (968